In [1]:
import pandas as pd
from tensorflow import keras
import tensorflow as tf


In [15]:
## Install and import the Keras Tuner.

In [2]:
!pip install -q -U keras-tuner

In [3]:
import keras_tuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
img_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0,   0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,
          1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          0,  36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,
          0,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,
          0, 102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,
         10,   0],
       [  

In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [7]:
img_train[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

**When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.**

In [8]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

**Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - 
RandomSearch,
Hyperband, 
BayesianOptimization, and 
Sklearn. 
In this tutorial, you use the Hyperband tuner.**

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

**Create a callback to stop training early after reaching a certain value for the validation loss.**

In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [11]:
tuner.search(img_train, label_train, epochs=5, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



**Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.**

In [12]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4982 - accuracy: 0.8230 - val_loss: 0.4118 - val_accuracy: 0.8541
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3714 - accuracy: 0.8643 - val_loss: 0.3737 - val_accuracy: 0.8668
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3304 - accuracy: 0.8789 - val_loss: 0.4085 - val_accuracy: 0.8505
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3092 - accuracy: 0.8856 - val_loss: 0.3324 - val_accuracy: 0.8778
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2874 - accuracy: 0.8928 - val_loss: 0.3350 - val_accuracy: 0.8773
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2729 - accuracy: 0.8985 - val_loss: 0.3275 - val_accuracy: 0.8808
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2601 - accuracy: 0.9034 - val_loss: 0.3281 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4995 - accuracy: 0.8223 - val_loss: 0.4012 - val_accuracy: 0.8573
Epoch 2/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3736 - accuracy: 0.8642 - val_loss: 0.3696 - val_accuracy: 0.8672
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3346 - accuracy: 0.8766 - val_loss: 0.3807 - val_accuracy: 0.8653
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3070 - accuracy: 0.8861 - val_loss: 0.3530 - val_accuracy: 0.8723
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2909 - accuracy: 0.8924 - val_loss: 0.3215 - val_accuracy: 0.8833
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2735 - accuracy: 0.8978 - val_loss: 0.3149 - val_accuracy: 0.8860
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2595 - accuracy: 0.9039 - val_loss: 0.3260 - val_accuracy:

  **evaluate the hypermodel on the test data**

In [14]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.5553 - accuracy: 0.8877
[test loss, test accuracy]: [0.5553167462348938, 0.8877000212669373]
